In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.1.3'
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://ppa.launchpad.net

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-10-12 19:33:13--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  4.98MB/s    in 0.2s    

2022-10-12 19:33:14 (4.98 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Mobile_Electronics_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Mobile_Electronics_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   20422322| R8MEA6IGAHO0B|B00MC4CED8|     217304173|BlackVue DR600GW-PMP|Mobile_Electronics|          5|            0|          0|   N|                Y|         Very Happy!|As advertised. Ev...| 2015-08-31|
|         US|   40835037|R31LOQ8JGLPRLK|B00OQMFG1Q|     137313254|GENSSI GSM / GPS ...|Mobile_Electronics|      

In [5]:
from pyspark.sql.functions import to_date, col, desc
# Read in the Review dataset as a DataFrame
df = df.dropna()
df.dtypes

[('marketplace', 'string'),
 ('customer_id', 'int'),
 ('review_id', 'string'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('product_title', 'string'),
 ('product_category', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string'),
 ('review_headline', 'string'),
 ('review_body', 'string'),
 ('review_date', 'string')]

In [10]:
# Create the vine_table DataFrame.
vine_df = df.select(["review_id", "star_rating","helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R8MEA6IGAHO0B|          5|            0|          0|   N|                Y|
|R31LOQ8JGLPRLK|          5|            0|          1|   N|                Y|
|R2Y0MM9YE6OP3P|          5|            0|          0|   N|                Y|
| RRB9C05HDOD4O|          4|            0|          0|   N|                Y|
|R26I2RI1GFV8QG|          2|            0|          0|   N|                Y|
| RY8DDL22YG4R5|          3|            0|          1|   N|                Y|
|R2AT2426ZHFUHH|          3|            0|          1|   N|                Y|
|R3RRXU2R23NMQ9|          5|            0|          0|   N|                Y|
|R250PR8VJUZ62F|          4|            0|          2|   N|                Y|
| RBEMQ29WJBHYG|          5|          164|        168|   N|     

In [12]:
# 1. Filter data and create DataFrame to collect all rows where total_votes is equal to or greater than 20. 
twenty_plus_votes_df = vine_df.filter("total_votes >=20")
twenty_plus_votes_df.orderBy(twenty_plus_votes_df["total_votes"].desc()).show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R15NZ47QWFG3AO|          5|          769|        791|   N|                Y|
|R2AJMFBMSZQ754|          3|          435|        448|   N|                Y|
| RWF03LXVXC22A|          3|          396|        445|   Y|                N|
|R203WXG4O3HOJP|          5|          425|        429|   N|                Y|
|R2ZE4SAM8HBB0Z|          1|          415|        427|   N|                N|
| R1Q1TFIZMVAQJ|          1|          323|        337|   N|                Y|
|R1E5YBZEWCBACE|          5|          310|        318|   N|                Y|
| R51C1SWZ7LR84|          5|          306|        314|   N|                Y|
|R2U5ZTNIJW03HV|          1|          257|        311|   N|                N|
|R2SW571Y0V66KI|          1|          293|        301|   N|     

In [14]:
# 2. Filter new DataFrame (twenty_plus_votes_df) to retrieve all rows where the number of helpful_votes / total_votes is equal to or greater than 50%.
helpful_votes_df = twenty_plus_votes_df.filter("(helpful_votes/total_votes)>=0.5")
helpful_votes_df.orderBy(helpful_votes_df["helpful_votes"].desc()).show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R15NZ47QWFG3AO|          5|          769|        791|   N|                Y|
|R2AJMFBMSZQ754|          3|          435|        448|   N|                Y|
|R203WXG4O3HOJP|          5|          425|        429|   N|                Y|
|R2ZE4SAM8HBB0Z|          1|          415|        427|   N|                N|
| RWF03LXVXC22A|          3|          396|        445|   Y|                N|
| R1Q1TFIZMVAQJ|          1|          323|        337|   N|                Y|
|R1E5YBZEWCBACE|          5|          310|        318|   N|                Y|
| R51C1SWZ7LR84|          5|          306|        314|   N|                Y|
|R2SW571Y0V66KI|          1|          293|        301|   N|                Y|
|R13W3EMIUV120L|          4|          288|        294|   N|     

In [23]:
# 3. Filter new DataFrame (helpful_votes_df) to create new DataFrame that retrieves all the rows where a review was written as part of the (paid) Vine program.
paid_reviews_df = helpful_votes_df.filter("vine == 'Y'")
paid_reviews_df.show(truncate=False)

+--------------+-----------+-------------+-----------+----+-----------------+
|review_id     |star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3BOQTLUU3Y4L8|4          |42           |55         |Y   |N                |
|RWF03LXVXC22A |3          |396          |445        |Y   |N                |
|REPU28WG1VZUE |5          |242          |281        |Y   |N                |
|R1D6REC9HPJVQY|4          |31           |41         |Y   |N                |
+--------------+-----------+-------------+-----------+----+-----------------+



In [24]:
# 4. Retrieve all the rows where the review was not part of the Vine program (unpaid).
unpaid_reviews_df = helpful_votes_df.filter("vine == 'N'")
unpaid_reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RBEMQ29WJBHYG|          5|          164|        168|   N|                Y|
|R13W3EMIUV120L|          4|          288|        294|   N|                Y|
|R30TJ8POYNVCRE|          5|           15|         21|   N|                N|
|R1LK03P7BCJEJD|          1|           31|         35|   N|                Y|
| RBXUICKGHL685|          1|           48|         52|   N|                N|
|R2E16WLZKP78GO|          5|           19|         21|   N|                Y|
|R1EN4PGQIFSENW|          5|           27|         28|   N|                Y|
| RXZWQU66AHIYG|          5|           24|         25|   N|                Y|
|R2IXCZCSUKH8FB|          3|           78|         82|   N|                Y|
|R25RB38U9LL8OE|          5|           24|         29|   N|     

In [25]:
# 5a. Determine the total number of reviews.
total_review_count_df = vine_df.count()
total_review_count_df

104972

In [40]:
# 5b. Determine the number of 5-star reviews.
five_stars_df = vine_df.filter("star_rating==5")
five_stars_count = five_stars_df.count()
five_stars_count

52253

In [42]:
# 5c. Determine the count of 5-star reviews for PAID (to find percentage).
paid_five_stars_df = five_stars_df.filter("vine == 'Y'")
paid_five_stars_count = paid_five_stars_df.count()
paid_five_stars_count

6

In [47]:
# 5d. Determine the percentage of 5-star reviews for PAID.
percent_paid_five_stars = (paid_five_stars_count/five_stars_count)*100
round(percent_paid_five_stars, ndigits=2)

0.01

In [48]:
# 5e. Determine the count of 5-star reviews for UNPAID (to find percentage).
unpaid_five_stars_df = five_stars_df.filter("vine == 'N'")
unpaid_five_stars_count = unpaid_five_stars_df.count()
unpaid_five_stars_count

52247

In [49]:
# 5f. Determine the percentage of 5-star reviews for UNPAID.
percent_unpaid_five_stars = (unpaid_five_stars_count/five_stars_count)*100
round(percent_unpaid_five_stars, ndigits=2)

99.99